# Data Preprocessing

In [1]:
#TODO authors regex regular expression
#TODO  dates


### lowerCase

In [2]:
def toLower(text):
    return text.lower()

### Numbers to words

In [3]:
import inflect
p = inflect.engine()

import re
reg = r'([0-9]+)'

def isFLoat(strNum):
    try:
        float(strNum)
        return True
    except:
        return False


def converteNumbers(text):
    tempText = text.split()
    newText = []
    for word in tempText:
        tempList = re.split(reg,word)
        for miniWord in tempList:
            if miniWord.isdigit() or isFLoat(miniWord):
                temp = p.number_to_words(miniWord)
                newText.append(removePunctuation(temp))
            else:
                newText.append(miniWord)        
    tempText = ' '.join(newText)
    return tempText

### remove punctuations

In [4]:
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
def removePunctuation(text):
    global translator
    return text.translate(translator)

### remove WhiteSpaces

In [5]:
def removeWhiteSpace(text):
    return " ".join(text.split())

### remove Stop Words

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def removeStopWords(text):
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)

### Stemming

In [7]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

def stemWords(text):
    global stemmer
    wt = word_tokenize(text)
    stems = [stemmer.stem(word) for word in wt]
    return ' '.join(stems)

### lemmatization

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatizeWords(text):
    wt = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos='v') for word in wt]
    return ' '.join(lemmas)

### data preprocessing

In [9]:
import pandas as pd
import re

caRegex = r'CA[0-9a-z]* ?[A-Z]+ '
def TitlePreProcesse(t):
    tempText = toLower(t)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def abstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def publicationPreProcesse(p):
    # tempText = toLower(p)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    # tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)

    # TODO remove cacm word and coverte the rest of the data to dateType 
    tempText = p.replace('CACM ','')
    return pd.to_datetime(tempText)

def authorPreProcesse(a):
    tempText = toLower(a)
    # tempText = converteNumbers(tempText)
    
    # TODO take the first wordbefore ,
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)

    # tempText = removePunctuation(tempText)
    
    # tempText = removeWhiteSpace(lis[0])
    
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return names

def kPreProcesse(k):
    tempText = toLower(k)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def cPreProcesse(c):
    # tempText = toLower(c)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(c)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

def infoPreProcesse(i):
    # tempText = toLower(i)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    #TODO remove the (CA581202 JB) ... and converte the rest to date type
    tempText = re.sub(caRegex, '', i)
    tempText = pd.to_datetime(tempText)
    #tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

In [10]:
import pandas as pd
def preprocessedData(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame(columns=['.I','data','.B','.C','.N','.X'])
    seriesDict:dict = {'.I':None, 'data':None, '.B':None, '.C':None,'.N':None, '.X':None} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            tempC = tempB = tempI = None
            if not dataFrame.loc[i, '.T'] == '':
                templist.append(TitlePreProcesse(dataFrame.loc[i, '.T']))
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(abstractPreProcesse(dataFrame.loc[i, '.W']))
            if not dataFrame.loc[i, '.B'] == '':
                tempB = publicationPreProcesse(dataFrame.loc[i, '.B'])
            if not dataFrame.loc[i, '.A'] == '':
                templist.append(authorPreProcesse(dataFrame.loc[i, '.A']))
            if not dataFrame.loc[i, '.K'] == '':
                templist.append(kPreProcesse(dataFrame.loc[i, '.K']))
            if not dataFrame.loc[i, '.C'] == '':
                tempC = cPreProcesse(dataFrame.loc[i, '.C'])
            if not dataFrame.loc[i, '.N'] == '':
                tempI = infoPreProcesse(dataFrame.loc[i, '.N'])

            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            seriesData['.B'] = tempB
            seriesData['.C'] = tempC
            seriesData['.N'] = tempI
            seriesData['.X'] = dataFrame.loc[i,'.X']
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [11]:
import pandas as pd
data = pd.read_csv('../../cacmData/cacmDataCleanedV1.csv')
data.set_index('.I', inplace=True)
data.fillna('', inplace=True)
data.head()

,.T,.W,.B,.A,.K,.C,.N,.X
.I,,,,,,,,
1,Preliminary Report-International Algebraic Lan...,,"CACM December, 1958","Perlis, A. J. Samelson, K.",,,CA581203 JB March 22 1978 8:28 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
2,Extraction of Roots by Repeated Subtractions f...,,"CACM December, 1958","Sugai, I.",,,CA581202 JB March 22 1978 8:29 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
3,Techniques Department on Matrix Program Schemes,,"CACM December, 1958","Friedman, M. D.",,,CA581201 JB March 22 1978 8:30 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
4,Glossary of Computer Engineering and Programmi...,,"CACM November, 1958",,,,CA581103 JB March 22 1978 8:32 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
5,Two Square-Root Approximations,,"CACM November, 1958","Wadey, W. G.",,,CA581102 JB March 22 1978 8:33 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."


In [12]:
d = preprocessedData(data)
d.head()

c:\Users\ASUS\anaconda3\envs\IRProject\lib\site-packages\dateutil\parser\_parser.py:1212: UnknownTimezoneWarning: tzname PMB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,data,.B,.C,.N,.X
.I,,,,,
1,preliminari report intern algebra languag perl...,1958-12-01,,1978-03-22 20:28:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
2,extract root repeat subtract digit comput sugai,1958-12-01,,1978-03-22 20:29:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
3,techniqu depart matrix program scheme friedman,1958-12-01,,1978-03-22 20:30:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
4,glossari comput engin program terminolog,1958-11-01,,1978-03-22 20:32:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
5,two squar root approxim wadey,1958-11-01,,1978-03-22 20:33:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."


In [13]:
l = d.loc[-5:, 'data'].tolist()
l[-10:]

['reiter acm polici toward standard period chang offic chairman editor usual follow elect occasion result chang polici case depart radic chang nevertheless proper time reiter an underlin acm polici respect standard comput area gorn',
 'reactiv typewrit program eighti four charact keyboard includ alphabet upper lower case good readabl machin restrict singl case lower case prefer reactiv typewrit portabl reactiv typewrit oper commerci use dial type telephon voic telegraph telex line leas nondial telegraph line interchang mooers',
 'structur standard process organ comput area line acm polici statement comm acm five nov one thousand nine hundr sixti two five hundr forti seven five hundr forti nine follow organiz descript provid order describ standard activ pertin comput inform process gorn bemer green',
 'microprogram emul program languag problem concern convert languag action intellectu energi mechan energi medium use purpos languag therefor preoccupi subject languag area languag investig

In [16]:
d.to_csv('../../cacmData/cacmDataPreprocessed.csv')